In [1]:
import MeCab
import CaboCha
import pandas as pd
from tqdm import tqdm 
import regex

In [2]:
def get_word(tree, chunk):
    surface = ''
    for i in range(chunk.token_pos, chunk.token_pos + chunk.token_size):
        token = tree.token(i)
        features = token.feature.split(',')
        if features[0] == '名詞':
            surface += token.surface
        elif features[0] == '形容詞':
            surface += features[6]
            break
        elif features[0] == '動詞':
            surface += features[6]
            break
    return surface

def get_2_words(line):
    cp = CaboCha.Parser('-f1')
    tree = cp.parse(line)
    chunk_dic = {}
    chunk_id = 0
    for i in range(0, tree.size()):
        token = tree.token(i)
        if token.chunk:
            chunk_dic[chunk_id] = token.chunk
            chunk_id += 1

    tuples = []
    for chunk_id, chunk in chunk_dic.items():
        if chunk.link > 0:
            from_surface =  get_word(tree, chunk)
            to_chunk = chunk_dic[chunk.link]
            to_surface = get_word(tree, to_chunk)
            tuples.append((from_surface, to_surface))
    return tuples

In [3]:
#口コミを１文ずつに切り分けし、その文章内を係り受け解析し（単語A,単語B）にkawaii_wordがあればリストに加える
def taples_kawaii(txt):
    kawaii_word = ['可愛い','かわいい','カワイイ','美人','綺麗','きれい','美しい','美女']
    sen = regex.split(r'(?<=[。？])(?!$)', txt, flags=regex.VERSION1)
    add_list=[]
    for i in sen:
        tuples = get_2_words(i)
        for j in tuples:
            for k in kawaii_word:
                if k in j:
                    add_list.append(j)
    return add_list

In [4]:
#形態素解析するための関数を定義
tagger = MeCab.Tagger('-Owakati')#タグはMeCab.Tagger（neologd辞書）を使用したい
tagger.parse('')
def tokenize_ja(text, lower):
    node = tagger.parseToNode(str(text))
    while node:
        if lower and node.feature.split(',')[0] in ["名詞","形容詞"]:#分かち書きで取得する品詞を指定
            yield node.surface.lower()
        node = node.next
def tokenize(content, token_min_len, token_max_len, lower):
    return [
        str(token) for token in tokenize_ja(content, lower)
        if token_min_len <= len(token) <= token_max_len and not token.startswith('_')
    ]

In [5]:
#kawaii_wordが含まれた（単語A,単語B）をさらに形態素解析し、tennin_wordが含まれていたらフラグ立てをする
def tennin_hantei(i):

    txt = tokenize(i, 2, 10000, True)

    vector_list =[]

    a = 0

    tennin_word = ['店員','スタッフ','女将','女性','店主','助手','奥さん','奥様','店長']
    for k in txt:
        if k in tennin_word:
            a+=1
        else:
            a+=0        
    if a >=1:
        kawaii_flg = 1
    else:
        kawaii_flg = 0

    return kawaii_flg

In [14]:
df = pd.read_csv('processed_all_results.csv')
df.head()
# df['score'].astype(float)
# df = df[float(df['score']) > 4.0]
# df.shape

,store_id,store_name,score,lunch_review,dinner_review
0,44,日本橋蛎殻町 すぎた,4.5,NaN,\n 2019年2月 再訪☆週末の夜、2回転目のすぎたさん。相変わらずのつまみに握り、安定的なおいしさのほかに、訪問するたびにハッとするおいしさ、発見がある。この日もつまみは追加で、日本酒が進みすぎてこまってしまう。日本酒泥棒★たこをやわらかく甘くたいたもの、すぎたさんで始めていただいたとき衝撃を受けたのを覚えている。タコってこんなにおいしいの？その時の気持ちがまた、再燃。たこ、おいしい～。固さと味のしみ方が本当に絶妙。握りも相変わらず安定的な抜群のおいしさ。ややおおきめの握り、かみしめて楽しむお鮨。美味しくて楽しい夜でした。。ごちそうさまでしたーおひとり様度：★★★迷子レベル：★☆☆リピ度：★★★予約難度：★★★（ほぼ不可）\n
1,44,日本橋蛎殻町 すぎた,4.5,NaN,\n 【つまみ】相変わらず酒の進む佳肴の数々だ。◉皮剥 肝醤油◉長崎壱岐産鰹塩をして寝かせた物を漬けにしている 浅葱と生姜を叩いた薬味で◉釧路仙鳳趾産牡蠣塩をした脱水させて旨味を凝縮した物で、ネットリと舌に絡みつく甘味と深いコクは正に海のミルクの代表選手だ。大根おろしの味付けも一手間加えた物で、醤油 山葵 酢橘 出汁を合わせてある。この大根薬味は見事なバランス感覚を保っていて仄かな山葵の辛味が隠し味で効いており、海のミルクの甘味を引き立て、かつクドく成らないようにブレーキ役も担っている。◉対馬産穴子白焼7〜8月の金沢八景の穴子が一番好みとのこと。本当の前浜産だと。小柴や子安は他の所にも行って獲っている◉雲子◉筋子味噌漬け◉鰤幽庵焼き一塩して寝かせた物を幽庵地に漬けて焼いている。塩をした分味が入ってしっとりとした感じを出したいと。【握り】煮切りは３種類を駆使してタネ...
2,44,日本橋蛎殻町 すぎた,5.0,NaN,\n この日は久々にかみさんと。「子供の面倒は？」という質問には黙秘権を使用させて頂きます。（決して悪いことはしておりません）いつも通り握りをガッツリとお願いしますが、さり気なく以下の摘みを供されます。・鮑、平目のお造り・数の子の味噌漬け・白子ぽん酢鮑は薄目に切りつけられ、握ってもいけそうな感じ。ゴロッとした切りつけとは変えてきました。旨味は十分で、滴る汁さえ勿体無いと感じるほど。平目も安定の一品。ほぼ寝かせずにお造りとして。数の子って私は正直そこまで美味しい食材だと思ったことはないのですが、こちらの数の子の味噌漬けは本当に旨い。ちびちび齧りつつ、追いかけるように酒を。白子ぽん酢、これもいいですね。口の中で濃厚なソースになります。他の方には鰤の漬けのお造りが。これが……すんごい旨そうで……きっと鰤は握りで出る！と思っておりましたが、予想は見事に外れました。鰤は次回...
3,44,日本橋蛎殻町 すぎた,5.0,NaN,\n 今回訪問したのは日本最高の評価を受けるすぎた。予約困難店で、15日の予約開始と共に4時間ぶっ通しで電話をかけ続けたところやっと繋がり、キャンセル待ちをお願いしたところ丁度キャンセルが出たとの事で訪問出来ました。食べログにはランチタイムは日曜日のみとありますが、土曜日もランチは営業しているようで12時からの一斉開始でスタートまずはつまみから蒸し鮑 舌触りがすごくなめらか。酒と塩でシンプルな味付けをされているようです。とり貝 食べたことがないぷるっとした食感が堪らなく美味しいです。しゃこ 北海道で食べて以来のしゃこ。肉っぽさがありみずみずしいほっき貝の生姜醤油焼き穴子の茶碗蒸しホタルイカ 味噌漬け ホタルイカの味がしっかり出ています。あん肝 甘辛いあん肝はいつ食べても美味しかった。日本酒とセットで頂きましたが相性抜群。太刀魚 つまみでい...
4,44,日本橋蛎殻町 すぎた,4.6,NaN,\n 蛎殻町である。水天宮を通り抜け、到着。同行者が車のため、駐車場は近くにたくさんあります。凛とした綺麗な空間で、暖簾をくぐり階段を降りると凄く天井が高いウェイティングスペース。手前に個室があり、奥に両サイドが斜めに向いた劇場型カウンター。ビールで乾杯です。•銀杏•カワハギと肝 ポン酢....この辺から味のハーモニーが凄くて口数が減り始めます。•甘く煮付けたあん肝新政の貴醸酒とマリアージュなのです。•鰹•鰯の巻物スペシャリテですが、1つ1つの高さを丁寧に揃える親方。•穴子の白焼き温度が抜群で、余韻がながーく口の中に残ります。•鰤お刺身も重厚で、ねっとりといつまでもいてくれる感じで大好きです。お酒は勝駒を。常温か冷や熱燗で種類を変えていただけます。•牡蠣•茶碗蒸し•白子•蛸•数の子それではシャリをつくってまいります と杉田親方。素敵です。•コハダ•赤身の漬け•エボ...


In [7]:
pd.set_option("display.max_colwidth", 400)
list_sentence = df['dinner_review'].tolist()

new_list=[]
for txt in tqdm(list_sentence):
    new_list.append(taples_kawaii(txt))
kawaii_flg_list2=[]
kawaii_flg_list3=[]
for k in new_list:
    kawaii_flg_list=[]
    if len(k)==0:
        kawaii_flg_list.append(0)
    else: 
        for i in k:
                kawaii_flg_list.append(tennin_hantei(i))
                if tennin_hantei(i)==1:
                    kawaii_flg_list3.append(sum(kawaii_flg_list))

df['kawaii_flg']=kawaii_flg_list3
df_ramen = df.groupby(['store_name','score'])['dinner_review'].apply(list).apply(' '.join).reset_index().sort_values('score', ascending=False)
df_ramen2 = df.groupby(['store_name','score'])['kawaii_flg'].sum().reset_index().sort_values('kawaii_flg', ascending=False)
df_ramen2

  0%|                                                                             | 15/17212 [00:10<2:34:03,  1.86it/s]

TypeError: expected string or buffer